In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from sklearn.neighbors import NearestNeighbors
import gc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#just helps you in getting started with the simplest submission file without too many steps. 
!pip install ../input/sentencetransformer/sentence-transformers-master

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('../input/sbertbasemodel/bert-base-nli-mean-tokens/')


In [ ]:
#taken from https://www.kaggle.com/ragnar123/shopee-inference-efficientnetb1-tfidfvectorizer/notebook#Comments
def get_neighbors(df, text_embeddings, KNN = 50):
    # Get distances and indices from image and text embeddings
    neighbors_model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine').fit(text_embeddings)
    text_distances, text_indices = neighbors_model.kneighbors(text_embeddings)
    predictions = []
    for k in range(df.shape[0]):
        # Reduce the thresholds because we are predicting more observations
        idx_text = np.where(text_distances[k,] < 0.05)[0]
        ids_text = text_indices[k,idx_text]
        # Get the union of boths ids
        ids = list(set(list(ids_text)))
        posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
        predictions.append(posting_ids)
    
    del neighbors_model, text_distances, text_indices
    gc.collect()
    return df, predictions

In [ ]:
df = pd.read_csv('../input/shopee-product-matching/test.csv')
df.title = df.title.str.lower()
embeddings = sbert_model.encode(df.title.values)

In [ ]:
if len(df) > 3:
    n = 50
else:
    n = 3

df, predictions = get_neighbors(df, embeddings, KNN = n)
df['matches'] = predictions
df[['posting_id', 'matches']].to_csv('submission.csv', index = False)
